In [ ]:
# PACKAGE IMPORTS #
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

from scipy import stats
from pprint import pprint

#### Create dataframe object from data in local file (.csv)

In [ ]:
def load_and_preprocess_data(filename, emails_req=True):
    """Load data from file and apply transformations"""
    #Load data from csv
    df = pd.read_csv(filename)
    
    #pre-process data cleaning
    df['date'] = pd.to_datetime(df['date']) # convert to datetime object
    df['week_num'] = df['date'].dt.week # get week number
    df['email'] = df['email'].dropna() # remove records with no email address
    
     
    #filter to relevant columns
#     col_list = [
#             'user_distinct_id', 
#             'email', 
#             'date',
#             'week_num',
#             'python_edit',
#             'completions_num_selected',
#         ]
#     df = df[col_list]
   
    return df

In [ ]:
df_ks1d_all = load_and_preprocess_data('Completions_Analysis_ks1d.csv')

##### Data Import Option: Get JQL query results directly from Mixpanel API

In [ ]:
#!pip install mixpanel-jql

#from datetime import datetime
#from mixpanel_jql import JQL, Reducer, Events

#mixpanel_api_secret = '...'

# query = JQL(
#             mixpanel_api_secret,
#             events=Events({
#                 'event_selectors': [{'event': "X"}],
#                 'from_date': datetime(2016, 5, 1),
#                 'to_date': datetime(2016, 5, 31)
#             })
#         ).group_by(
#             keys=[
#                 "new Date(e.time).toISOString().split('T')[0]",
#             ],
#             accumulator=Reducer.count()
#         )

### Build dataframe from mixpanel-jql query which streams back results
#for row in query.send():
#  df

#### Analyze kite_status_1d events in Dataframe

In [ ]:
#dataframe for ks1d events including all results from JQL query
df = df_ks1d_all.copy()

df.head()

In [ ]:
# example of how to create new dataframe objects that contain only desired rows, cols
df_recent = df.loc[df['date'] > '2019-09-01']

#df_recent.head()

In [ ]:
#df.loc[df['email'] == 'caelan@kite.com'].sort_values('python_edit', ascending=False)

In [ ]:
# Example of computing new stats and append a new column to the dataframe
#df['ratio_compl_triggered_to_pyedit'] = df.completions_triggered / df.python_edit
#df.head()

### Visualization tools

In [ ]:
# Add Plotly

#### Python_edit total for a prior calendar period

In [ ]:
#df.loc[:,['date', 'python_edit']][df.date < '2019-10-21']

In [ ]:
# use datetime library for relative time operations
now = datetime.now()
one_week = timedelta(days=7)

one_week_ago = now - one_week
print(one_week_ago.month, one_week_ago.day)

In [ ]:
# Dataframe with events from last week only
df_lastweek = df.loc[:,['email','date', 'python_edit']][df.date > '2019-10-21']

# Dataframe with events from this calendar year-to-date
df_ytd = df.loc[:,:][df.date >= '2019-01-01']

## Counts by week per user (email)

In [ ]:
df_weekly_totals = df.groupby(['email', 'week_num']).sum()
df_weekly_totals

## Counts Year-to-Date per user

In [ ]:
df_ytd_totals = df_ytd.groupby(['email']).sum()
#df_ytd_totals

## Counts for 2019

In [ ]:
#Dataframe with all data from 2019
df_2019 = df.loc[:,:][(df.date >= '2019-01-01') & (df.date <= '2019-12-31')]

In [ ]:
df_2019.describe()

### _How does completions_selected trend daily over 2019?_

#### Outlier Analysis & Removal

In [ ]:
df_2019.plot(y='completions_num_selected', x='date', title="Completions Selected 2019")
# This plot shows abnormal spikes on a few dates in Feb, June, July, and Aug

In [ ]:
outlier_rows = df_2019[(stats.zscore(df_2019['completions_num_selected']) > 5)]
outlier_dates = pd.Series(outlier_rows['date'].unique()).dt.date.sort_values()

outlier_dates.apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

The list of outlier dates was obtained by this method:
- For each column, first it computes the Z-score of each value in the column, 
relative to the column mean and standard deviation.
- Then it takes the absolute of Z-score because the direction does not matter.
- Z-score must be within 5 (i.e., value not more than 5 standard deviations from col mean)
- Below, the result is used to index a new dataframe designated "cleaned"

In [ ]:
df_2019_cleaned = df_2019[(np.abs(stats.zscore(df_2019['completions_num_selected'])) < 5)]

In [ ]:
df_2019_cleaned

#### 2019 Dataframe with outlier rows removed

In [ ]:
df_2019_cleaned.plot(y='completions_num_selected', x='date', title="Completions Selected 2019")

In [ ]:
df_2019_cleaned['completions_num_selected'].describe()

In [ ]:
# Sum all activity for each date to a single aggregate value, for each column
date_grouped_df = df_2019_cleaned.groupby('date').sum()

In [ ]:
# Rolling 14 day average completions_num_selected
date_grouped_df['completions_num_selected'].rolling(window=14).mean().plot(title="Rolling 14 Day Average completions_num_selected")

In [ ]:
# Rolling 14 day average completions_num_selected
date_grouped_df['completions_num_selected'].rolling(window=1).max().plot(title="Daily max completions_num_selected")

In [ ]:
# Note: likely more individual outliers to remove from Feb, Jul, Aug based on above chart

#### _What’s the total number of completions_selected by all Kite users in 2019?_

In [ ]:
print("Total completions_num_selected in 2019: ", df_2019_cleaned['completions_num_selected'].sum())
# approx. 70 million before / 50 million after removing some outliers

## Q4 2019
Ranking users by completion feature usage

In [ ]:
#Dataframe with all data from Q4 2019 only
df_q42019 = df_2019_cleaned.loc[:,:][(df_2019_cleaned.date >= '2019-10-01') & (df_2019_cleaned.date <= '2019-12-31')]

### python_edit top ranked users

In [ ]:
df_q42019 = df_q42019.sort_values('python_edit', ascending=False)

In [ ]:
#Top 10 users ranked by highest single day python_edit total
df_q42019.head(10)

In [ ]:
#Top 10 users ranked by highest python_edit total (sum of user's activity for dates in Q4)
df_q42019.groupby('email').sum().sort_values('python_edit', ascending=False).head(10)

In [ ]:
#Just these users' email addresseses as a rank ordered list
df_q42019.groupby('email').sum().sort_values('python_edit', ascending=False).index.tolist()

### completions_num_selected top ranked users

In [ ]:
#Top 10 users ranked by highest completions_num_selected total (sum of user's activity for dates in Q4)
df_q42019.groupby('email').sum().sort_values('completions_num_selected', ascending=False).head(10)

In [ ]:
#The top 100 users' email addresseses as a rank ordered list
top_user_emails = df_q42019.groupby('email').sum().sort_values('completions_num_selected', ascending=False).index.tolist()[:100]
for email in top_user_emails:
    print(email)